# Dataset report: ChemBL


> https://www.ebi.ac.uk/chembl/

In [1]:
import altair as alt
alt.data_transformers.disable_max_rows()
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

import cytoxnet.dataprep.io

## Start with the MIC query

In [2]:
## get the data
dataset = cytoxnet.dataprep.io.load_data('chembl_ecoli_MIC')
dataset.describe()

,ecoli_MIC,molecular_weight
count,5274.000000,5274.000000
mean,-inf,529.441943
std,NaN,368.933762
min,-inf,61.040000
25%,1.832581,356.360000
50%,3.465736,435.550000
75%,4.158883,546.582500
max,9.433484,4462.640000


In [3]:
print('Number of unique molecules: ', len(dataset))

Number of unique molecules:  5274


### Targets present

#### <span style='color:blue'>__The range of targets seems to be quite large for each species (units of ug/mL)__</span>

In [4]:
dataset.describe().loc[['min', 'max']]

,ecoli_MIC,molecular_weight
min,-inf,61.04
max,9.433484,4462.64


In [6]:
alt.Chart(dataset).mark_area(
    opacity=0.7,
    interpolate='step'
).encode(
    alt.X('ecoli_MIC:Q', bin=alt.Bin(maxbins=100)),
    alt.Y('count()', stack=None)
)

alt.Chart(...)

#### <span style='color:blue'>__The dataset is heavily imbalanced towards the toxic side (units log ug/mL)__</span>

### Molecule space

In [20]:
!pip install --quiet umap-learn hdbscan

distutils: /Users/ek/miniconda3/envs/cytoxnet/include/python3.6m/UNKNOWN
sysconfig: /Users/ek/miniconda3/envs/cytoxnet/include/python3.6m
user = False
home = None
root = None
prefix = None
distutils: /Users/ek/miniconda3/envs/cytoxnet/include/python3.6m/UNKNOWN
sysconfig: /Users/ek/miniconda3/envs/cytoxnet/include/python3.6m
user = False
home = None
root = None
prefix = None


In [39]:
import rdkit.Chem.AllChem
import umap

Set the descriptors to use for mapping

In [40]:
dataset['descriptor'] = dataset['smiles'].apply(
    lambda smiles: rdkit.Chem.AllChem.GetMorganFingerprintAsBitVect(rdkit.Chem.MolFromSmiles(smiles), radius=2, nBits=2048)
    )

UMAP the smiles

In [41]:
%%time
umap_model = umap.UMAP(metric = "jaccard",
                      n_neighbors = 25,
                      n_components = 2,
                      low_memory = False,
                      min_dist = 0.001)
X_umap = umap_model.fit_transform(np.vstack(dataset['descriptor'].values))
dataset["UMAP_0"], dataset["UMAP_1"] = X_umap[:,0], X_umap[:,1]

/Users/ek/miniconda3/envs/cytoxnet/lib/python3.6/site-packages/umap/umap_.py:1762: UserWarning: gradient function is not yet implemented for jaccard distance metric; inverse_transform will be unavailable
  "inverse_transform will be unavailable".format(self.metric)


CPU times: user 1min 6s, sys: 391 ms, total: 1min 6s
Wall time: 39.7 s


Are there any clusters?

In [44]:
alt.Chart(dataset[['UMAP_0', 'UMAP_1']]).mark_circle(size=60).encode(
    x='UMAP_0',
    y='UMAP_1',
)

alt.Chart(...)

#### <span style='color:blue'>__All three species seem to cover the available space__</span>

### Do any clusters in UMAP space seem to exhibit high toxicity?

In [46]:
alt.Chart(dataset[['UMAP_0', 'UMAP_1', 'LD50']]).mark_circle(size=60).encode(
    x='UMAP_0',
    y='UMAP_1',
    color='LD50:Q',
)

alt.Chart(...)

#### <span style='color:blue'>__There is a small cluster of very non toxic molecules__</span>